In [37]:
%%writefile plug_ai.slurm
#!/bin/bash
#SBATCH --job-name=plug_ai
#SBATCH --output=plug_ai.out
#SBATCH --error=plug_ai.out
#SBATCH --gres=gpu:1
#SBATCH --ntasks-per-node=1
#SBATCH --nodes=1
#SBATCH --hint=nomultithread
#SBATCH --time=00:50:00
#SBATCH --qos=qos_gpu-dev
#SBATCH --cpus-per-task=10
#SBATCH --account=sos@v100
#SBATCH -C v100-16g


## load Pytorch module
module purge

module load pytorch-gpu/py3/1.10.1

## launch script on every node
set -x

# code execution
srun python ../plug_ai/__main__.py --config_file "../config_exemples/config_Brats.yaml"

Overwriting plug_ai.slurm


In [38]:
%%bash
# submit job
sbatch plug_ai.slurm

Submitted batch job 1381859


In [39]:
from threading import Event
import signal
from IPython.display import clear_output

def quit(signo, _frame):
    print("Interrupted by %d, shutting down" % signo)
    exit.set()

for sig in ('TERM', 'HUP', 'INT'):
    signal.signal(getattr(signal, 'SIG'+sig), quit);

exit = Event()

sq = !squeue -u $USER -n plug_ai
tail = !tail plug_ai.out
while len(sq) >= 2 and not exit.is_set():
    
    clear_output(wait=True)
    print(sq[0], sq[1], sep='\n')
    print(*tail, sep='\n')
    
    sq = !squeue -u $USER -n plug_ai
    tail = !tail plug_ai.out

    exit.wait(1)

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1381859   gpu_p13  plug_ai  ssos022  R       1:29      1 r10i4n8
[Epoch 1/1 |  Step_Epoch 2/8 | Loss 3.040724992752075]
[Epoch 1/1 |  Step_Epoch 3/8 | Loss 3.0586326122283936]
[Epoch 1/1 |  Step_Epoch 4/8 | Loss 3.02146053314209]
[Epoch 1/1 |  Step_Epoch 5/8 | Loss 3.0172150135040283]
[Epoch 1/1 |  Step_Epoch 6/8 | Loss 2.9689812660217285]
[Epoch 1/1 |  Step_Epoch 7/8 | Loss 2.981983184814453]
[Epoch 1/1 |  Step_Epoch 8/8 | Loss 2.988492965698242]
Epoch 0 finished
Execution over
>>> Complete execution in: 0:00:23.562164


In [9]:
!scancel -u $USER -n plug_ai